## Letting `DataTool` index local data
In this example notebook, we will point the `DataTool` class towards our locally stored images and allow it to index them, relative to an input list of source coordinates.

Firstly, in case this repository is cloned and not pip installed...

In [ ]:
from sys import path
import os, glob

# Get the current working directory of the notebook
notebook_dir = os.getcwd()

# Navigate up to the project root
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))

# Add the project root to sys.path
if project_root not in path:
    path.append(project_root)

Now import the necessary modules, classes and functions for this notebook

In [2]:
from quest.lib_data import DataTool
from quest.lib_query import QueryTool, default_online_archive_config
from quest.lib_photometry import PhotometryTool
from quest.lib_plot import plot_multiwl_stamps, plot_overlays, plot_photometry, plot_all_stamps
import warnings
from astropy.io import fits
from astropy.wcs import FITSFixedWarning
from astropy.coordinates import SkyCoord
import astropy.units as u

Here is an example configuration that feeds in to the initialisation and use of the classes. Some directories are created if not already existing, but they should already match the directory structure of the repository. Feel free to swap out values to test with your own data and input source lists.

In [3]:
# --- Global Configuration ---
# Working path, input source list, output database file
data_dir = '../demo/data'
src_list_file = f'{data_dir}/example_sources.lis'
# Input source list can be parsed as you wish later, but it must contain:
#   - Source (long) name... typically the J2000 coordinate in hms dms
#   - Source (short) name... this is used for display/annotation on plots
#   - Source RA and Dec... can be in whatever format you wish, but it will have to be made into a SkyCoord instance
db_save_path = f'{data_dir}/source_data.db'

# Output directories for figures and data
output_fig_dir = '../demo/figures'
output_data_dir = '../demo/output'
if not os.path.exists(output_fig_dir):
    os.mkdir(output_fig_dir)
if not os.path.exists(output_data_dir):
    os.mkdir(output_data_dir)

# Optional to section off your data into directories in some fashion (e.g. wavelength domain).
# These must exist in the working path (e.g. `./demo/data`, creating them if not).
data_sub_dirs = [f'{data_dir}/Optical', f'{data_dir}/IR', f'{data_dir}/Radio']
for search_dir in data_sub_dirs:
    if not os.path.exists(search_dir):
        os.mkdir(search_dir)

# You can define the **PHOTOMETRY** search paths here.
# Searches are made at the level of the defined paths below recursively (e.g. /my/path/**/*.fits)
data_dir_list = [f'{data_dir}/Optical', f'{data_dir}/IR']
# Similar for the **CONTOUR** search paths here, which we use the radio for (only tested with radio currently)
data_dir_radio = [f'{data_dir}/Radio']

# You can define the paths to ignore when searching here
# (i.e. if a searched path CONTAINS any of the strings in the ignore_paths list).
ignore_paths = ['dud_cutouts', f'{data_dir}/IR/unWISE', f'{data_dir}/IR/2MASS', f'{data_dir}/Radio/TGSS']

# Filter out certain warnings
warnings.filterwarnings('ignore', category=FITSFixedWarning)
warnings.filterwarnings('ignore', category=fits.verify.VerifyWarning)

Now we get into using the `DataTool`, initialising from a saved .db file if it exists, otherwise starting fresh. We will parse our list of input sources and scan the search directories for images that contain the sources. We save the state of the `DataTool` and its constituent `DataEntry` and `SourceEntry` instances, which could be saved to a *unique name* as a **checkpoint** if you want to revert the state of the `DataTool` later on.

NOTE: since this code was written with obtaining optical/IR data for known radio sources in mind, the "input source list" is sometimes referred to as a list of input radio coordinates. This does not actually matter, since the input coordinates are merely the initial coordinates (you could call them "approximate" or "guess" coordinates if you don't care about radio!).

For the purpose of this notebook, we have run it with only a couple images for each source already downloaded (not included in the repository). For now, read through the structure of this notebook and then move on to `defining_and_retrieving_archive_data.ipynb`, where you can re-download some of the data and possibly come back here to give it a spin!

In [4]:
##### Main Workflow #####

# 1. Initialize or load the data database
try:
    db = DataTool()
    db.load_from_file(filepath=db_save_path)
except (FileNotFoundError, EOFError):
    print("No existing database found or file is empty, initializing a new one.")
    db = DataTool(search_directories=data_dir_list)
    db.set_ignored_filepaths(ignore_paths=ignore_paths)

# 2. Add sources from the list file (here we parse the ra and dec assuming the input list is in decimal degrees)
with open(src_list_file, 'r') as f:
    for line in f:
        source_name, display_name, ra, dec = line.split(',')
        coord = SkyCoord(float(ra), float(dec), unit='deg')
        # This will now check the new source against all known images
        db.add_source(source_name, display_name, coord)

# 3. Scan local directories for new images
# This will now check any new images against all known sources
db.scan_local_directories()

db.save_to_file(db_save_path)

No existing database found or file is empty, initializing a new one.
Adding new source: TN J0924-2201. Checking against 0 known images.
Adding new source: VIK J2318−3113. Checking against 0 known images.
Scanning local directories for FITS files...
Found 4 new images. Checking against 2 sources.
Adding ../demo/data/IR/VISTA/23h18m18.35s-31d13m46.34s_VIKING_Ks_180.0arcsec_cutout.fits	(1/4)                                                  
  -> Found new 'Ks' data for VIK J2318−3113 in VISTA/23h18m18.35s-31d13m46.34s_VIKING_Ks_180.0arcsec_cutout.fits (RMS: 5.00e-04)
Adding ../demo/data/Optical/DECaLS/09h24m19.92s-22d01m41.50s_z_180.0arcsec_cutout.fits	(2/4)                                                  
  -> Found new 'z' data for TN J0924-2201 in DECaLS/09h24m19.92s-22d01m41.50s_z_180.0arcsec_cutout.fits (RMS: 9.83e-08)
Adding ../demo/data/Optical/DECaLS/23h18m18.35s-31d13m46.34s_z_180.0arcsec_cutout.fits	(3/4)                                                  
  -> Found new 'z' data

### Indexing held data finished!
Now we can move on to the next example notebook: `defining_and_retrieving_archive_data.ipynb`, or we can inspect the data-driven properties that are stored in the `DataTool` from indexing our local images.

In [5]:
# We can inspect the properties of individual DataEntry instances that are associated with one of our input sources.
key1 = list(db.sources[0].best_data.keys())[0]
print(f"Band: {key1}")
print(db.sources[0].best_data[key1])
# or for every one of our sources, we can check a property of the best data for a certain wavelength band!
for src in db.sources:
    de = src.best_data.get(key1)
    if de:
        print(src.source_name, de.instrument, de.band, de.wavelength, de.rms)

Band: z
DataEntry(self.filepath='../demo/data/Optical/DECaLS/09h24m19.92s-22d01m41.50s_z_180.0arcsec_cutout.fits'
        		self.astronomy_method='photometric'
        		self.instrument='DECaLS'
        		self.band='z'
        		self.wavelength=<Quantity 0.926 um>
        		self.rms=9.8251164e-08
        		self.pix_res=<Quantity 7.27777778e-05 deg>)
			
TN J0924-2201 DECaLS z 0.926 um 9.8251164e-08
VIK J2318−3113 DECaLS z 0.926 um 4.3266343e-08


In [6]:
# Another example of leveraging the data-driven properties: Print all the unique instruments used.
unique_instruments = set()
for source in db.sources:
    for data in source.best_data.values():
        unique_instruments.add(data.instrument)

print(f"Unique instruments used in best data: {unique_instruments}")

Unique instruments used in best data: {'VIRCAM', 'DECaLS'}


In [7]:
# Yet another example: Count the number of sources that contain data from a specific instrument
# (only count once per source if at least one band is from that instrument)
count = 0
instrument = db.sources[0].best_data[key1].instrument
for src in db.sources:
    if any(data.instrument == instrument for data in src.best_data.values()):
        count += 1
print(f'Number of sources with at least one {instrument} observation: {count}')

Number of sources with at least one DECaLS observation: 2


In [8]:
# Although the output can get very long, you can also inspect the full breakdown of a SourceEntry and
# its associated `best_data` DataEntry instances.
db.sources[0]

SourceEntry(self.source_name='TN J0924-2201'
        	self.short_label=' J0924'
        	self.radio_coord=<SkyCoord (ICRS): (ra, dec) in deg
    (141.083, -22.028194)>
        	self.host_coord=<SkyCoord (ICRS): (ra, dec) in deg
    (141.083, -22.028194)>
        	self.best_data={'z': DataEntry(self.filepath='../demo/data/Optical/DECaLS/09h24m19.92s-22d01m41.50s_z_180.0arcsec_cutout.fits'
        		self.astronomy_method='photometric'
        		self.instrument='DECaLS'
        		self.band='z'
        		self.wavelength=<Quantity 0.926 um>
        		self.rms=9.8251164e-08
        		self.pix_res=<Quantity 7.27777778e-05 deg>)
			, 'Ks': DataEntry(self.filepath='../demo/data/IR/VISTA/09h24m19.92s-22d01m41.50s_VHS_Ks_180.0arcsec_cutout.fits'
        		self.astronomy_method='photometric'
        		self.instrument='VIRCAM'
        		self.band='Ks'
        		self.wavelength=<Quantity 2.146 um>
        		self.rms=0.0007193797960999267
        		self.pix_res=<Quantity 9.48188594e-05 deg>)
			})
	